In [1]:
import pandas as pd
import cv2
import os
import numpy as np

# Creating and Appending in csv file

In [2]:
#Creating CSV file
'''
raw_data = {'Name':[], 'ID':[], 'Path':[], 'Vec1':[], 'Vec2':[], 'Vec3':[], 'Vec4':[], 'Vec5':[]}
df = pd.DataFrame(raw_data, columns = ['Name', 'ID', 'Path', 'Vec1', 'Vec2', 'Vec3', 'Vec4', 'Vec5'])
df.to_csv('Sample.csv')
df
'''

"\nraw_data = {'Name':[], 'ID':[], 'Path':[], 'Vec1':[], 'Vec2':[], 'Vec3':[], 'Vec4':[], 'Vec5':[]}\ndf = pd.DataFrame(raw_data, columns = ['Name', 'ID', 'Path', 'Vec1', 'Vec2', 'Vec3', 'Vec4', 'Vec5'])\ndf.to_csv('Sample.csv')\ndf\n"

In [3]:
def Appending(Name,Num,Path,num1,num2,num3,num4,num5):
    ds = pd.read_csv('Sample.csv',index_col=0)
    length = len(ds)
    #print(length)
    ds.loc[length+1] =[Name,Num,Path,num1,num2,num3,num4,num5]
    ds.to_csv('Sample.csv')

# Taking 5 photos of the user 

In [5]:
def createfolder(name) :
    newpath = 'Your Directory Path'+name 
    if not os.path.exists(newpath):
        os.makedirs(newpath)

In [6]:
def FivePhotos(num) :
    cam = cv2.VideoCapture(0)

    cv2.namedWindow("FivePhotos")

    img_counter = 1

    while True:
        ret, frame = cam.read()
        #cv2.imshow("FivePhotos", frame)
        if not ret:
            break
        face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            
        cv2.imshow("FivePhotos", frame)
            
        k = cv2.waitKey(1)

        if img_counter == 6:
            print("Thank's for your time.")
            break
        elif k%256 == 32: #Space
            img_name = "opencv_frame_{}.jpeg".format(img_counter)
            path = 'Your Directory Path'+str(num)
            #cv2.imwrite(img_name, frame)
            print(FaceOrNot(frame))
            if FaceOrNot(frame) == True :
                createfolder(str(num))
                cv2.imwrite(os.path.join(path , img_name), frame)
                img_counter += 1

    cam.release()

    cv2.destroyAllWindows()
    
    return path


# we have captured a face ?

In [7]:
def FaceOrNot(img) :
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    #print(faces)
    if len(faces) == 1 :
        return True
    else :
        return False

# Vector Extraction

In [8]:
#Model Calling
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Layer
from model import create_model
import numpy as np
import os.path

Using TensorFlow backend.


In [9]:
nn4_small2_pretrained = create_model()
nn4_small2_pretrained.load_weights('weights/nn4.small2.v1.h5')

In [10]:
#Input Preprocessing
import cv2
from align import AlignDlib

%matplotlib inline

def load_image(path):
    img = cv2.imread(path, 1)
    #print(img)
    # OpenCV loads images with color channels
    # in BGR order. So we need to reverse them
    return img[...,::-1]

# Initialize the OpenFace face alignment utility
alignment = AlignDlib('landmarks.dat')

def align_image(img):
    return alignment.align(96, img, alignment.getLargestFaceBoundingBox(img), 
                           landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)

In [11]:
#Extraction Phase
def ListOfVec(ID) : 
    Resultlist = []
    for i in range(5) :
        path = str(ID) + "/opencv_frame_" + str(i+1) + ".jpeg"
        #print(path)
        img = load_image(path)
        img = align_image(img)
        img = (img / 255.).astype(np.float32)
        Resultlist.append(nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0])
        #print(len(Resultlist))
    return Resultlist

# Main 

In [14]:
def UniqueID(ID):
    dsx = pd.read_csv('Sample.csv',index_col=0)
    saved_column = list(dsx['ID'])
    if ID in saved_column :
        return False
    else :
        return True

In [17]:
Name = input("Enter your name please : ")
while(True) :
    Num = input("Enter your ID please : ")
    if UniqueID(int(Num)) == True : 
        break 
    else :
        print("Enter a unique ID. ")
print("Press Space to take a photo , Please Take 5 Photos")
Path = FivePhotos(Num)
VecList = []
VecList = ListOfVec(Num)
Appending(Name,Num,Path,VecList[0],VecList[1],VecList[2],VecList[3],VecList[4])

Enter your name please : Jimmy
Enter your ID please : 7896
Press Space to take a photo , Please Take 5 Photos
False
False
False
False
False
False
True
False
False
False
False
True
False
False
False
False
False
False
False
True
True
True
Thank's for your time.
